# Overlap comparisons

In this notebook we investigate the overlap of the VHS and HSC Wide surveys in order to estimate computing times and data storage requirements.

In [1]:
from astropy.table import Table
import astropy.units as u
import json
import glob

In [14]:
hsc_patches = json.loads(open('hsc_total_patch_dict.json', 'r').read())
vhs_patches = json.loads(open('vhs_total_patch_dict.json', 'r').read())

In [7]:
hsc_patches

{'17131': [[2, 7],
  [7, 2],
  [8, 1],
  [7, 0],
  [5, 7],
  [3, 8],
  [8, 3],
  [3, 4],
  [8, 6],
  [3, 7],
  [8, 8],
  [2, 5],
  [7, 6],
  [5, 5],
  [5, 4],
  [6, 1],
  [4, 7],
  [3, 6],
  [4, 5],
  [8, 5],
  [6, 0],
  [8, 0],
  [2, 6],
  [7, 4],
  [8, 7],
  [6, 8],
  [4, 6],
  [4, 8],
  [8, 4],
  [8, 2],
  [6, 7],
  [7, 3],
  [4, 4],
  [5, 3],
  [7, 1],
  [7, 8],
  [6, 3],
  [2, 8],
  [7, 5],
  [7, 7],
  [6, 2],
  [6, 5],
  [5, 8],
  [6, 6],
  [3, 5],
  [5, 6],
  [6, 4]],
 '17272': [[7, 2],
  [5, 2],
  [8, 4],
  [7, 0],
  [8, 2],
  [8, 1],
  [7, 3],
  [7, 1],
  [6, 3],
  [8, 3],
  [6, 0],
  [6, 2],
  [5, 0],
  [6, 1],
  [8, 0],
  [5, 1],
  [7, 4]],
 '8765': [[2, 7],
  [1, 5],
  [0, 6],
  [8, 1],
  [0, 1],
  [3, 8],
  [8, 6],
  [8, 8],
  [2, 4],
  [2, 5],
  [7, 6],
  [4, 3],
  [3, 0],
  [1, 0],
  [1, 7],
  [5, 4],
  [1, 4],
  [4, 7],
  [3, 6],
  [8, 0],
  [7, 4],
  [1, 1],
  [8, 7],
  [1, 3],
  [4, 6],
  [4, 0],
  [8, 4],
  [0, 7],
  [8, 2],
  [7, 3],
  [5, 3],
  [7, 1],
  [0, 3],
  

In [13]:
n=0
for tract in hsc_patches:
    n+=len(hsc_patches[tract])
print("There are {} patches in HSC which is {} square degrees of processing area (including overlap).".format(
    n,
    round(n*((1.7*u.deg/9)**2).value)
))

There are 14926 patches in hsc which is 533.0 square degrees of processing area (including overlap)


In [14]:
n=0
for tract in vhs_patches:
    n+=len(vhs_patches[tract])
print("There are {} patches in VHS which is {} square degrees of processing area (including overlap).".format(
    n,
    round(n*((1.7*u.deg/9)**2).value)
))

There are 670137 patches in VHS which is 23910.0 square degrees of processing area (including overlap)


In [21]:
hsc_vhs_overlap = {}

for tract in hsc_patches:
    
    try:
        hsc_vhs_overlap[tract] = set(["[{},{}]".format(p[0],p[1]) for p in hsc_patches[tract]]).intersection(
            set(["[{},{}]".format(p[0],p[1]) for p in vhs_patches[tract]])
        )
    except KeyError:
        pass
    
for tract in hsc_vhs_overlap:
    hsc_vhs_overlap[tract] = [[int(p[1]),int(p[3])] for p in hsc_vhs_overlap[tract]]

In [23]:
json.dumps(hsc_vhs_overlap, separators=(',', ':'))

'{"8765":[[3,4],[8,5],[1,7],[0,1],[1,8],[3,5],[8,1],[1,5],[4,5],[2,0],[8,8],[3,3],[2,6],[5,4],[6,3],[3,8],[6,1],[6,6],[0,3],[2,4],[1,3],[1,4],[2,7],[1,1],[5,1],[0,6],[7,7],[2,5],[0,4],[5,0],[2,2],[1,6],[5,8],[4,3],[6,2],[2,1],[3,2],[6,5],[2,3],[5,3],[6,0],[1,2],[4,0],[8,6],[3,0],[7,4],[7,0],[1,0],[6,7],[0,7],[4,1],[7,5],[6,4],[8,4],[7,1],[0,5],[0,2],[0,0],[5,7],[4,6],[5,5],[7,6],[4,2],[0,8],[4,4],[3,7],[5,6],[7,2],[3,6],[8,3],[8,7],[8,2],[7,8],[4,8],[7,3],[5,2],[6,8],[4,7],[8,0],[2,8],[3,1]],"9465":[[8,6],[8,8],[8,4],[6,8],[8,3],[8,7],[6,7],[8,5],[8,2],[5,8],[7,8],[7,6],[7,5],[5,7],[7,7],[4,8],[6,6]],"9706":[[1,0],[3,4],[5,1],[4,2],[0,6],[0,1],[4,4],[2,5],[7,0],[0,4],[5,0],[3,5],[4,1],[2,2],[1,5],[4,5],[3,6],[1,6],[2,0],[3,3],[2,6],[0,5],[3,1],[0,2],[4,3],[5,4],[0,0],[6,2],[6,3],[3,2],[2,1],[6,1],[4,6],[5,2],[0,3],[2,3],[5,3],[5,5],[6,0],[2,4],[1,3],[1,2],[4,0],[8,0],[1,4],[3,0],[1,1]],"9572":[],"8525":[[3,4],[6,7],[8,5],[7,6],[1,7],[7,7],[4,4],[2,5],[3,7],[1,8],[3,5],[8,1],[5,6],[7,2]

In [24]:
with open('hsc_vhs_overlap.json', 'w') as outfile:
    json.dump(hsc_vhs_overlap, outfile, separators=(',', ':'))

In [26]:
n=0
for tract in hsc_vhs_overlap:
    n+=len(hsc_vhs_overlap[tract])
print("There are {} patches in VHS and HSC which is {} square degrees of processing area (including overlap).".format(
    n,
    round(n*((1.7*u.deg/9)**2).value)
))

There are 2040 patches in VHS and HSC which is 73.0 square degrees of processing area (including overlap).


# TODO DOWNLOAD ALL HSC FILES - THIS NUMBER IS TOO LOW!!!

For now lets get an upper bound by just taking the tracts defined in the field definition tract lists

In [8]:
wide_tract_files = glob.glob('../dmu0/dmu0_HSC/tracts_patches/*_W*.txt')
wide_tract_files

['../dmu0/dmu0_HSC/tracts_patches/tracts_patches_W-AEGIS.txt',
 '../dmu0/dmu0_HSC/tracts_patches/tracts_patches_W-w06.txt',
 '../dmu0/dmu0_HSC/tracts_patches/tracts_patches_W-w04.txt',
 '../dmu0/dmu0_HSC/tracts_patches/tracts_patches_W-w05.txt',
 '../dmu0/dmu0_HSC/tracts_patches/tracts_patches_W-w01.txt',
 '../dmu0/dmu0_HSC/tracts_patches/tracts_patches_W-w02.txt',
 '../dmu0/dmu0_HSC/tracts_patches/tracts_patches_W-w03.txt']

In [12]:

tracts = set()

for tract_file in wide_tract_files:
    f = open(tract_file, 'r')
    for x in f:
        if x.startswith('Tract'):
            tracts.add(int(x.split(' ')[1])) 
len(tracts)

911

In [19]:
vhs_tracts = set()
for t in vhs_patches:
    vhs_tracts.add(int(t))
len(vhs_tracts)

8743

In [20]:
tract_overlap = vhs_tracts.intersection(tracts)
len(tract_overlap)

333

In [21]:
print("There are {} tracts in VHS and HSC which is {} square degrees of processing area (including overlap).".format(
    len(tract_overlap),
    round(len(tract_overlap)*81*((1.7*u.deg/9)**2).value)
))

There are 333 tracts in VHS and HSC which is 962.0 square degrees of processing area (including overlap).


In [1]:
333 * (1.7**2)


962.3699999999999